### 加载数据集

In [14]:
from utils import load_corpus, stopwords

TRAIN_PATH = "./data/weibo_dataset/train.txt"
TEST_PATH = "./data/weibo_dataset/test.txt"

In [15]:
# 分别加载训练集和测试集
train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

In [16]:
import pandas as pd

df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_test = pd.DataFrame(test_data, columns=["words", "label"])
df_train.head()

,words,label
0,书中 自有 黄金屋 书中 自有 颜如玉 沿着 岁月 的 长河 跋涉 或是 风光旖旎 或是 姹...,1
1,这是 英超 被 黑 的 最惨 的 一次 二哈 二哈 十几年来 中国 只有 孙继海 董方卓 郑...,0
2,中国 远洋 海运 集团 副总经理 俞曾 港 月 日 在 上 表示 中央 企业 走 出去 是 ...,1
3,看 流星花园 其实 也 还好 啦 现在 的 观念 以及 时尚 眼光 都 不一样 了 或许 十...,1
4,汉武帝 的 罪己 诏 的 真实性 尽管 存在 着 争议 然而 轮台 罪己 诏 作为 中国 历...,1


### TF-IDF向量化文本数据

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 向量化文本数据
tfidf_vectorizer = TfidfVectorizer(
    token_pattern=r'(?u)\b\w\w+\b',  # 匹配两个及以上字符的词
    stop_words=stopwords
)
X_train = tfidf_vectorizer.fit_transform(df_train["words"])
y_train = df_train["label"]

In [18]:
X_test = tfidf_vectorizer.transform(df_test["words"])
y_test = df_test["label"]

### 定义逻辑回归模型，训练、预测

In [19]:
from sklearn.linear_model import LogisticRegression

# 定义并训练逻辑回归模型
model = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    C=1.0,
    class_weight='balanced'  # 自动调整类别权重
)

model.fit(X_train, y_train)

# 预测并评估
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # 获取正类概率，用于计算AUC

In [20]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred, digits=3))
print("准确率:", metrics.accuracy_score(y_test, y_pred))
print("AUC:", metrics.roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0      0.764     0.729     0.746       155
           1      0.881     0.899     0.890       345

    accuracy                          0.846       500
   macro avg      0.822     0.814     0.818       500
weighted avg      0.844     0.846     0.845       500

准确率: 0.846
AUC: 0.9154184198223468
